In [31]:
# PGM intro

In [8]:
from pgmpy.factors.discrete import TabularCPD
from pgmpy.models import BayesianModel

In [9]:
#set a basic structure
wc_model = BayesianModel([('Genetics', 'Performance'),('Practice','Performance'), ('Performance','Offer')])

In [10]:
#set up relationship
genetics_cpd = TabularCPD(variable ='Genetics', variable_card = 2, values = [[.2,.8]])

In [11]:
practice_cpd = TabularCPD(variable ='Practice', variable_card = 2, values = [[.7,.3]])

In [12]:
offer_cpd = TabularCPD(variable ='Offer', 
                       variable_card = 2, 
                       values = [[.95,.8,.5],
                                 [.05, .2, .5]],
                      evidence = ['Performance'],
                      evidence_card = [3])

In [13]:
wc_trials_cpd = TabularCPD(variable ='Performance', 
                           variable_card = 3,
                           values = [[.5, .8, .8, .9],
                                     [.3, .15, .1, .08],
                                     [.2, .05, .1, .02]],
                      evidence = ['Genetics', 'Practice'],
                      evidence_card = [2,2])

In [14]:
# add the relationships to the model
wc_model.add_cpds(genetics_cpd, practice_cpd, offer_cpd, wc_trials_cpd)

In [15]:
# examine the structure of the graph
wc_model.get_cpds()

[<TabularCPD representing P(Genetics:2) at 0x9c6cf60>,
 <TabularCPD representing P(Practice:2) at 0x9c6cb38>,
 <TabularCPD representing P(Offer:2 | Performance:3) at 0x9c6c470>,
 <TabularCPD representing P(Performance:3 | Genetics:2, Practice:2) at 0x9c940b8>]

In [16]:
# find active trail nodes
wc_model.active_trail_nodes('Genetics')

{'Genetics': {'Genetics', 'Offer', 'Performance'}}

In [20]:
# find local independencies
wc_model.get_independencies()

(Genetics _|_ Practice)
(Genetics _|_ Offer | Performance)
(Genetics _|_ Offer | Performance, Practice)
(Practice _|_ Genetics)
(Practice _|_ Offer | Performance)
(Practice _|_ Offer | Performance, Genetics)
(Offer _|_ Genetics, Practice | Performance)
(Offer _|_ Practice | Performance, Genetics)
(Offer _|_ Genetics | Performance, Practice)

In [21]:
# making inferences
# we can get probability distributions that are not explicitely spelled out in our graphs
from pgmpy.inference import VariableElimination


In [24]:
wc_infer = VariableElimination(wc_model)

In [27]:
prob_offer = wc_infer.query(variables = ['Offer'])
print(prob_offer['Offer'])

╒═════════╤══════════════╕
│ Offer   │   phi(Offer) │
╞═════════╪══════════════╡
│ Offer_0 │       0.8898 │
├─────────┼──────────────┤
│ Offer_1 │       0.1102 │
╘═════════╧══════════════╛


In [30]:
prob_offer_bad_genes = wc_infer.query(variables = ['Offer'],
                                     evidence = {'Genetics':1})
print(prob_offer_bad_genes['Offer'])

╒═════════╤══════════════╕
│ Offer   │   phi(Offer) │
╞═════════╪══════════════╡
│ Offer_0 │       0.9017 │
├─────────┼──────────────┤
│ Offer_1 │       0.0983 │
╘═════════╧══════════════╛
